# WSW HDOP Check

Iterate through folders, looking for possible spikes in all supported GPS file types.

Copyright 2024 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

## Notes

This is very crude and simply looks for Doppler speeds in excess of 30 m/s (i.e. 108 km/h or 58.32 knots).

Such speeds are currently unrealistic for a windsurfer.

In [61]:
import os
import sys

import numpy as np

from shapely.geometry import Polygon, Point

import traceback

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Main Function

In [62]:
def runChecks():
    '''Iterate through session archive testing each GPS file'''

    rootDir = os.path.join(projdir, '..', 'wsw-data')

    errors = {}
    spikes = {}
    
    totFiles = 0
    totPoints = 0
    totDist = 0

    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            #if ext and ext in ['.sbn', '.sbp', '.ubx']:
            if ext and ext in ['.oao']:
                filePath = os.path.join(root, file)
                reader = getFileReader(filePath)
                try:
                    # Some legacy ESP-GPS files contain bad checksums
                    if ext == '.ubx':
                        reader.load(ignoreChecksums=True)
                    else:
                        reader.load()
                    
                    # Process all tracks within the file
                    for track in reader.tracks:
                        maxSog = 0

                        points = (np.argwhere(track.data['hdop'] > 5))
                        for i in points:
                            if track.data['fix'][i[0]] > 0 and \
                                    track.data['ehve'][i[0]] <= 1.2 * 1852 / 3600:
                                sog =  track.data['sog'][i[0]]
                                if sog > 0:
                                    print("fix={},sog={:.02f},hdop={:.02f},ehpe={:.02f},ehve={:.02f}".format(
                                        track.data['fix'][i[0]],  track.data['sog'][i[0]], track.data['hdop'][i[0]],
                                        track.data['ehpe'][i[0]],  track.data['ehve'][i[0]]))
                                    if sog > maxSog:
                                        maxSog = sog
                    
                        if maxSog > 0:
                            msg = '{:.02f},{:.02f},{:.02f}'.format(
                                maxSog, maxSog * 3600 / 1852, maxSog * 3600 / 1000)
                            spikes[filePath.replace(projdir + '/', '')] = msg
                            print('H', end='')

                        print('.', end='')
                        
                        totPoints += len(track.data['sog'])
                        totDist += track.data['sog'].sum()

                    totFiles += 1

                except Exception:
                    errors[filePath.replace(projdir + '/', '')] = traceback.format_exc()
                    print('E', end='')

    print('\nSummary: files = {}, points = {}, distance = {:,.02f} km'.format(totFiles, totPoints, totDist / 1000))

    if len(errors) > 0:
        print(os.linesep * 2 + 'Errors:')
        for error in errors:
            print(error)
            print(errors[error])

    if len(spikes) > 0:
        print(os.linesep * 2 + 'Crashes:')
        for spike in spikes:
            print(spike + ',' + spikes[spike])

In [63]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    runChecks()
    
    print(os.linesep + 'All done!')

.....................................................................................
fix=3,sog=3.21,hdop=10.52,ehpe=3.79,ehve=0.57
H......................................................
fix=3,sog=0.32,hdop=5.79,ehpe=33.50,ehve=0.53

fix=3,sog=0.23,hdop=5.69,ehpe=5.85,ehve=0.61

fix=3,sog=0.90,hdop=5.69,ehpe=5.65,ehve=0.61

fix=3,sog=0.03,hdop=6.56,ehpe=21.50,ehve=0.22

fix=3,sog=0.05,hdop=6.56,ehpe=21.61,ehve=0.30
H.......................
fix=3,sog=2.58,hdop=6.89,ehpe=12.07,ehve=0.61

fix=3,sog=2.61,hdop=13.35,ehpe=11.35,ehve=0.60

fix=3,sog=2.75,hdop=13.35,ehpe=11.34,ehve=0.60

fix=3,sog=2.80,hdop=13.34,ehpe=11.35,ehve=0.61

fix=3,sog=2.86,hdop=13.34,ehpe=11.35,ehve=0.60

fix=3,sog=2.70,hdop=15.92,ehpe=10.78,ehve=0.58

fix=3,sog=2.59,hdop=15.92,ehpe=10.42,ehve=0.57

fix=3,sog=2.60,hdop=15.95,ehpe=9.13,ehve=0.56

fix=3,sog=2.64,hdop=15.95,ehpe=8.94,ehve=0.56

fix=3,sog=2.61,hdop=15.95,ehpe=8.81,ehve=0.55

fix=3,sog=2.64,hdop=7.60,ehpe=8.74,ehve=0.55

fix=3,sog=2.59,hdop=7.60,ehpe=8.5